# Homework 2

This homework will have a mix of theory questions and coding questions defined throughout the notebook. Please turn the theory questions in at class on Friday, Sept. 20, and upload the notebook to canvas.

In [ ]:
# This is code for loading the libraries needed for the assignment
import numpy as np
from casadi import *
import scipy.linalg as la
import matplotlib.pyplot as plt
import minecraftControl as mc

env = mc.constrainedBall()

%matplotlib inline

# Constrained Sphere Gym Environment

The rolling sphere problem from last week has been modified to be compatible with OpenAI Gym, a popular testbed for reinforcement learning. This is so that our codes for the class can be streamlined for compatibility. 

This problem is similar to the system we used last week, except now constraints on the state and input are enforced.

An example of its use, with randomly generated inputs is given below.

In [ ]:
# We need to first call reset to initialize the state
x = env.reset()
X_traj = [x]
U_traj = []


for _ in range(1000):
    # Randomly generate a feasible input
    u = env.action_space.sample()
    x,r,done,info = env.step(u)
    
    # Saving the state and action sequences
    X_traj.append(x)
    U_traj.append(u)
    
    # Render shows the video
    # If you don't want video, comment the line out.
    env.render()
    if done:
        # This stops the simulation when 
        # a constraint is violated
        print(info)
        print('u:',u)
        print('x:',x)
        break
    
env.close()

# Plot the trajectories
X_traj = np.array(X_traj)
U_traj = np.array(U_traj)
fig,ax = plt.subplots(2,1)
ax[0].plot(X_traj)
ax[0].set_ylabel('x')
ax[1].plot(U_traj)
ax[1].set_ylabel('u')

In [ ]:
# The constraints are shown here
print('x - lower bound:',env.observation_space.low)
print('x - upper bound:',env.observation_space.high)
print('u - lower bound:',env.action_space.low)
print('u - upper bound:',env.action_space.high)

This homework will step through the procedure of building an MPC controller for this system 

The dynamics of the system are given by
$$
x^+ = Ax + Bu
$$
where $A$ and $B$ are the matrices defined below.

In [ ]:
dt = env.dt
A = np.array([[1,0,3 *dt,0],
              [0,1,0,3 * dt],
              [0,0,1,0],
              [0,0,0,1]])

B = np.array([[0,0],
              [0,0],
              [dt,0],
              [0,dt]])

# Theory Question 1 



Recall that the infitinite-horizon regulator solution for step cost $x^\top Q x + u^\top R u$
is given by $u= Kx$ where $K$ is defined by:
\begin{align*}
S&= Q + A^\top S A - A^\top S B(R+B^\top S B)^{-1} B^\top S A \\
K&= -(R+B^\top S B)^{-1} B^\top S A
\end{align*}

Assume that $S$ is positive definite and the origin is asymptoically stable for the system $x^+=(A+BK)x$.
Show that $V(x) = x^\top S x$ is a Lyapunov function for this system. 

Furthermore,  show that if $x(k+1) = (A+BK)x(k)$ and $\ell(x,u) = x^\top Q x + u^\top R u$,  then
\begin{equation*}
x(0)^\top S x(0) = \sum_{k=0}^{\infty} \ell(x(k),u(k))
\end{equation*}

# Coding Question 1

Let $Q$ be and $R$ be identity matrices of appropriate size. Use the command `la.solve_discrete_are` to find $S$, and then compute $K$ for this system.

In [ ]:
# Define your solution here

# Change these to the correct values
S = np.zeros((4,4))
K = np.zeros((2,4))

# Theory Question 2

One of the fundamental conditions required to ensure stability of MPC is that the final constraint set $\mathbb{X}_f$ and the final cost $V_f$ be such that for all $x\in \mathbb{X}_f$, there is a $u$ such
*  $Ax+Bu\in \mathbb{X}_f$ (control invariance)
* $V_f(Ax+Bu) + \ell(x,u) \le V_f(x)$ (non-increasing end cost)
* $(x,u)$ is feasible (control feasibility).

We saw that if the system is linear, and a stabilizing control gain $K$, then the final cost and constraint could be constructed from the corresponding Lyapunov function. Namely, if 
$$
V_f(x) = x^\top Sx,
$$
where $S$ and $K$ are defined as above, and $\mathbb{X}_f = \{x | V_f(x) \le c\}$ for some positive $c$, then the control invariance and non-increasing cost conditions are automatically satisfied by choosing $u=Kx$.

Thus, the only thing that remains is to find $c>0$ such that the feasibility condition holds. Here we will describe a systematic method for finding such a $c$.

The constraints on the control and state were defined above. They can be written in vector form 
$$
Fx + Gu \preceq h,
$$
where the inequality is interpreted elementwise.

Show that $c$ can be found by solving a collection of quadratic programs. (A quadratic program is an optimization problem in which the objective is quadratic and the constraints are affine.)

Explain how the code below implements such a procedure. (Of course, this makes the question easier, but CasADi has substantially less online documentation than other tools used in this class. So seeing how it is used for our problems is helpful.)

In [ ]:
# NOTE: This code will only work properly if K is a stabilizing gain and S is the corresponding
# Lyapunov Matrix. In other words, you need to have solved the previous problem.

# Here is CasADi syntax for defining symbolic variables
x_sym = MX.sym('x',4)

u_sym = K @ x_sym

# Unlike numpy, CasADi assumes everything is a matrix
# So, you need to do explicit transposes to represent row vecrors
QuadObj = x_sym.T @ S @ x_sym

# These will represent constraints to be violated
g_u_low = env.action_space.low - u_sym
g_u_high = u_sym - env.action_space.high

g_x_low = env.observation_space.low - x_sym
g_x_high = x_sym - env.observation_space.high

g_vec = vertcat(g_u_low,g_u_high,g_x_low,g_x_high)

c = np.inf

for i in range(g_vec.shape[0]):
    qp = {'x' : x_sym,'f' : QuadObj, 'g' : g_vec[i]}

    solver = qpsol('solver','qpoases',qp)
    qp_res = solver(lbg=0)
    print(qp_res['f'])
    c = np.min([qp_res['f'],c])
    
# Just to make it a bit conservative, decrease a bit further
c = .9 * c

# Coding Question 2

Use CasADi to create a nonlinear program for computing an MPC controller using the costs $\ell(x,u) = x^\top x + u^\top u$, and $V_f(x) = x^\top S x$. Because the final cost constraint $V_f(x_N)\le c$ is quadratic, the problem can no longer be cast as quadratic program. So, you'll need to use the CasADi nonlinear programming interface. (The syntax is similar to the quadratic program.)

With $\mathbb{X}_f$ and $c$ defined as above, you'll likely need a horizon of around $N=50$ to ensure that the MPC problem is always feasible. 

Test your controller by simulating 100 steps of the gym environment and plotting the result. (So, just modify the code above.)

Hint: Due to numerical error, your MPC controller may produce inputs that slightly violate the constraints, which will cause the simulation to stop. To get around this, you can "tighten"  the constraints by casting them in the form $Fx + Gx\le h -\epsilon \mathbf{1}$, where $\epsilon $ is a small positive number and $\mathbf{1}$ is the vector of all ones.

In [ ]:
# Make your code here